<a href="https://colab.research.google.com/github/Shreyabansal3112/twitter-sentiment-analysis/blob/main/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV

import nltk
from nltk import corpus
from nltk.stem.porter import PorterStemmer
import io

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving movie_data.csv to movie_data.csv


In [ ]:
df = pd.read_csv("movie_data.csv")

In [ ]:
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [ ]:
docs = np.array(['The sun is shining',
                 'The weather is sweet',
                 'THe sun is shining, the weather is sweet, and one and one is two']) 

In [ ]:
count = CountVectorizer()

bag = count.fit_transform(docs)

In [ ]:
count.vocabulary_

{'and': 0,
 'is': 1,
 'one': 2,
 'shining': 3,
 'sun': 4,
 'sweet': 5,
 'the': 6,
 'two': 7,
 'weather': 8}

In [ ]:
bag.toarray()

array([[0, 1, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 1],
       [2, 3, 2, 1, 1, 1, 2, 1, 1]])

In [ ]:
np.set_printoptions(precision=2)

In [ ]:
tfidf = TfidfTransformer(use_idf = True, norm='l2', smooth_idf = True)
fitted = tfidf.fit_transform(bag)


In [ ]:
fitted.toarray()

array([[0.  , 0.43, 0.  , 0.56, 0.56, 0.  , 0.43, 0.  , 0.  ],
       [0.  , 0.43, 0.  , 0.  , 0.  , 0.56, 0.43, 0.  , 0.56],
       [0.5 , 0.45, 0.5 , 0.19, 0.19, 0.19, 0.3 , 0.25, 0.19]])

In [ ]:
import re
def prep(text):
    text = re.sub('<[^>]*>','', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +        ' '.join(emoticons).replace('-','')
    return text

In [ ]:
prep(df.loc[0, 'review'][-50:])


'is seven title brazil not available'

In [ ]:
df['review'] = df['review'].apply(prep)

In [ ]:
inpp = "this is a overall good movie but its bit too complex"
inpp = prep(inpp)


In [ ]:
porter = PorterStemmer()

In [ ]:
def tokenizer(text):
    return text.split()

In [ ]:
def tokenizer_porter(text):  #tokenizer_stem
    return [porter.stem(word) for word in text.split()]



In [ ]:
tokenizer_porter("runners like running and thus they run")

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords

stop = stopwords.words('english') # using the words of english
[w for w in tokenizer_porter('a runner like running and thus they runs lots') if w not in stop]



['runner', 'like', 'run', 'thu', 'run', 'lot']

In [ ]:

inppp = [w for w in tokenizer_porter(inpp) if w not in stop]



In [ ]:
inpp = np.array([inpp])

In [ ]:
tfidf = TfidfVectorizer(strip_accents=None,
                       lowercase=False,
                       preprocessor=None,
                       tokenizer=tokenizer_porter,
                       use_idf=True,
                       norm='l2',
                       smooth_idf=True)

y = df.sentiment.values
x = tfidf.fit_transform(df.review)

In [ ]:
type(x)
x

<50000x73814 sparse matrix of type '<class 'numpy.float64'>'
	with 6785262 stored elements in Compressed Sparse Row format>

In [ ]:
import pickle

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1,
                                                    test_size=0.5,
                                                   shuffle=False)






In [ ]:
clf = LogisticRegressionCV(cv=5,
                           scoring='accuracy',
                           max_iter=300,
                           n_jobs=-1,
                           verbose=3,
                           random_state=0).fit(x_train,y_train)

saved_model = open('saved_model.sav','wb')
pickle.dump(clf, saved_model)
saved_model.close()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.5min finished


In [ ]:
filename = 'saved_model.sav'
saved_clf = pickle.load(open(filename, 'rb')) # rb - read bytes

In [ ]:
saved_clf.score(x_test, y_test)

0.89608

In [ ]:
def custom(text):
    text = re.sub('<[^>]*>','', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +    ' '.join(emoticons).replace('-','')
    
    text = np.array([text])

    bag1 = count.fit_transform(text)
    np.set_printoptions(precision=2)

    tfidf = TfidfTransformer(use_idf = True, norm='l2', smooth_idf = True)
    fitted = tfidf.fit_transform(bag1)
    return fitted


In [ ]:
 text = np.array(["in 1974 the teenager martha moxley maggie grace moves to the high class area of belle haven greenwich connecticut on the mischief night eve of halloween she was murdered in the backyard of her house and her murder remained unsolved twenty two years later the writer mark fuhrman christopher meloni who is a former la detective that has fallen in disgrace for perjury in o j simpson trial and moved to idaho decides to investigate the case with his partner stephen weeks andrew mitchell with the purpose of writing a book the locals squirm and do not welcome them but with the support of the retired detective steve carroll robert forster that was in charge of the investigation in the 70 s they discover the criminal and a net of power and money to cover the murder murder in greenwich is a good tv movie with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a kennedy the powerful and rich family used their influence to cover the murder for more than twenty years however a snoopy detective and convicted perjurer in disgrace was able to disclose how the hideous crime was committed the screenplay shows the investigation of mark and the last days of martha in parallel but there is a lack of the emotion in the dramatization my vote is seven title brazil not available"])


In [ ]:
custom_test = custom("in 1974 the teenager martha moxley maggie grace moves to the high class area of belle haven greenwich connecticut on the mischief night eve of halloween she was murdered in the backyard of her house and her murder remained unsolved twenty two years later the writer mark fuhrman christopher meloni who is a former la detective that has fallen in disgrace for perjury in o j simpson trial and moved to idaho decides to investigate the case with his partner stephen weeks andrew mitchell with the purpose of writing a book the locals squirm and do not welcome them but with the support of the retired detective steve carroll robert forster that was in charge of the investigation in the 70 s they discover the criminal and a net of power and money to cover the murder murder in greenwich is a good tv movie with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a kennedy the powerful and rich family used their influence to cover the murder for more than twenty years however a snoopy detective and convicted perjurer in disgrace was able to disclose how the hideous crime was committed the screenplay shows the investigation of mark and the last days of martha in parallel but there is a lack of the emotion in the dramatization my vote is seven title brazil not available")


In [ ]:
type(custom_test)


scipy.sparse.csr.csr_matrix

In [ ]:
custom_test

<1x139 sparse matrix of type '<class 'numpy.float64'>'
	with 139 stored elements in Compressed Sparse Row format>

In [ ]:
x_train

<25000x73814 sparse matrix of type '<class 'numpy.float64'>'
	with 3388430 stored elements in Compressed Sparse Row format>

In [ ]:
clf.predict(custom_test)

ValueError: ignored